In [1]:
# %pip freeze > requirements.txt

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [31]:
# from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from jinja2 import Template
from langchain.agents import AgentExecutor, create_react_agent

from dotenv import load_dotenv
import json
import os
import re
import json
import random

In [32]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [105]:
JSON_FILE_PATH = "..\\use_openai\\data\\id_added_dummy_data.json"
# RCMD_PROMPT_PATH = "..\\prompts\\agent_recommend.md"
RCMD_PROMPT_PATH = "..\\prompts\\react_prompt.md"  # ReAct에 맞는 프롬프트로 수정
FEATURE_ID_PATH = "featureId.txt"

In [106]:
# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()

# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(type(data))  # (test) <class 'list'>

# Read featureId keys
with open(FEATURE_ID_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()
    feature_id_list = [line.strip() for line in lines]
    # print(feature_id)  # test

<class 'list'>


In [7]:
# ##### test #####
# FEATURE_ID = int(random.choice(feature_id_list))
# print(FEATURE_ID, type(FEATURE_ID))

In [107]:
# featureId로 해당 category, feature 이하의 actions만 추출
FEATURE_ID = int(random.choice(feature_id_list))

def extract_workspace_view(input_data, id):
    # featureId로 작업목록 + workspaceId 가져오기
    for item in input_data:
        for category in item.get("recommendedCategories", []):
            for feature in category.get("features", []):
                if feature.get("featureId") == id:
                    return {
                        "workspaceId": item["workspaceId"],
                        "recommendedCategories": [
                            {
                                "categoryId": category["categoryId"],
                                "name": category["name"],
                                "features": [feature]
                            }
                        ]
                    }

# featureId로 읽어온 actions
workspace_data = extract_workspace_view(data, FEATURE_ID)
workspace_data
# print(json.dumps(result, ensure_ascii=False, indent=2))

if workspace_data:
    print(json.dumps(workspace_data, ensure_ascii=False, indent=2))
else:
    print("해당 featureId를 찾을 수 없습니다.")

{
  "workspaceId": 9,
  "recommendedCategories": [
    {
      "categoryId": 4,
      "name": "게시판 기능",
      "features": [
        {
          "featureId": 75,
          "name": "게시글 관리",
          "actions": [
            {
              "name": "게시글 신고",
              "importance": 1,
              "startDate": "2025-06-19T14:25:34.094132",
              "endDate": "2025-06-21T14:25:34.094132"
            },
            {
              "name": "게시글 검색",
              "importance": 4,
              "startDate": "2025-06-20T14:25:34.094135",
              "endDate": "2025-07-04T14:25:34.094135"
            }
          ]
        }
      ]
    }
  ]
}


In [108]:
type(workspace_data)  # (test) dict

dict

In [109]:
# featureId로 읽어온 actions
workspace_data = extract_workspace_view(data, FEATURE_ID)

# 프롬프트 template 객체 생성. 템플릿 구조상(복잡, dict 생성이 필요) jinja2로 렌더링 필요.
template = Template(RCMD_PROMPT)

# 템플릿&작업목록 데이터 결합. jinja2 방식 템플릿 렌더링.
# template 내 변수({})들을 실제값으로 치환 -> 문자열 생성
# formatted_input = template.render(ws_data=workspace_data)  # JSON 내 필드만 하나씩 꺼내서 출력
formatted_input = template.render(ws_data=json.dumps(workspace_data, ensure_ascii=False))   #  JSON 전체를 문자열로 템플릿에 삽입

print(formatted_input)  # test
print(type(formatted_input))

# LangChain ReAct Agent 프롬프트 (JSON 작업 흐름 기반 액션 추천용)

당신은 JSON 작업 흐름을 분석하여, 각 기능(feature)에 대해 이후에 자연스럽게 이어질 작업(action) 3개씩을 추천하는 에이전트입니다.

다음은  기능의 기존 작업들입니다:



각 추천 작업은 반드시 다음 네 가지 필드를 포함해야 합니다:
- `name`: 작업 이름
- `startDate`: 시작 시점 (ISO 8601 형식, 예: YYYY-MM-DDTHH:MM:SS)
- `endDate`: 종료 시점 (startDate보다 늦어야 함)
- `importance`: 중요도 (1~5 사이의 정수, 값이 클수록 중요한 작업임)

결과는 반드시 **순수 JSON 형식**으로 출력해야 하며, **설명, 마크다운, 부가 텍스트 없이** 결과만 보여주어야 합니다.

응답은 아래와 같은 형식이어야 합니다:

```json
{
  "workspaceId": "",
  "recommendedCategories": [
    {
      "categoryId": "",
      "name": "",
      "startDate": "YYYY-MM-DDTHH:MM:SS",
      "endDate": "YYYY-MM-DDTHH:MM:SS",
      "importance": 5,
      "features": [
        {
          "featureId": "",
          "name": "",
          "startDate": "YYYY-MM-DDTHH:MM:SS",
          "endDate": "YYYY-MM-DDTHH:MM:SS",
          "importance": 1,
          "actions": [
            {
              "name": "",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDat

In [110]:
type(template)  # jinja2.environment.Template

jinja2.environment.Template

In [111]:
# 변수{} 삽입 가능한 템플릿 생성
prompt = PromptTemplate(
    input_variables=["featureNname", "existingActions", "tools", "tool_names", "agent_scratched"],
    template=RCMD_PROMPT
    # template="""
    # 다음은 '{featureName}' 기능의 기존 작업들입니다:

    # {existingActions}

    # 이 작업의 다음 단계로서, 자연스럽게 이어질 수 있는 작업 3개를 추천하세요.
    # 각 작업은 다음 항목을 포함해야 합니다:

    # - name: 구체적 작업명
    # - startDate: YYYY-MM-DDTHH:MM:SS 형식
    # - endDate: YYYY-MM-DDTHH:MM:SS 형식
    # - importance: 1~5 정수(값이 클수록 중요한 작업임)

    # 형식은 반드시 JSON 배열 형태로 출력하세요.
    # 예시:
    # ```json
    # [
    #   {
    #     "name": "...",
    #     "startDate": "...",
    #     "endDate": "...",
    #     "importance": 3
    #   },
    #   ...
    # ]
    # ```
    # """
)

In [112]:
##### test #####
search = DuckDuckGoSearchRun()
conv_data = json.dumps(workspace_data, ensure_ascii=False)  # dict -> str
pattern = r'({[\s\S]*})'
match = re.search(pattern, conv_data, re.DOTALL)
json_block = match.group(0)
print(type(json_block))
print(json_block)

<class 'str'>
{"workspaceId": 9, "recommendedCategories": [{"categoryId": 4, "name": "게시판 기능", "features": [{"featureId": 75, "name": "게시글 관리", "actions": [{"name": "게시글 신고", "importance": 1, "startDate": "2025-06-19T14:25:34.094132", "endDate": "2025-06-21T14:25:34.094132"}, {"name": "게시글 검색", "importance": 4, "startDate": "2025-06-20T14:25:34.094135", "endDate": "2025-07-04T14:25:34.094135"}]}]}]}


In [113]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [114]:
# llm(OpenAI) <-> Langchain 연결
# llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain = LLMChain(prompt=prompt, llm=llm)

# LLM으로 작업 추천 받는 함수 -> 여기서 프롬프트까지 받아서 작업 생성
def generate_next_actions(input_dict: dict) -> str:
    return llm_chain.run(input_dict)


# 전체 JSON을 받아 feature별 추천 작업을 생성하는 메인 함수 -> 여기서는 데이터만 받고
def parse_and_generate_all(input_str: dict) -> str:

    data = input_str  # 수정!!!!!!!

    # 웹 검색 툴 정의
    search = DuckDuckGoSearchRun()

    for category in data.get("recommendedCategories", []):  # recommendedCategories 가져오기
        for feature in category.get("features", []):        # recommendedCategories 내부의 feature에서

            # 웹 검색
            try:
                search_query = f"{feature['name']} 기능 구현 방법"  # feature name에 대한 actions 서치
                search_result = search.run(search_query)
            except Exception as e:
                search_result = f"** 웹 검색 실패: {e} **"

            # LLM 입력 구성 (웹검색 결과 포함)
            # llm_input = {
            #     "featureName": feature["name"],
            #     "existingActions": [a["name"] for a in feature.get("actions", [])],
            #     "webSearch": search_result[:500]  # 길이 제한
            # }
            llm_input = {search_result[:500]}

            try:
                generated_json_str = generate_next_actions(llm_input)  # 추천 actions 생성
                generated_actions = json.loads(generated_json_str)     # 추천 actions를 json형식의 python 객체로 load
                feature["actions"].extend(generated_actions)
            except Exception as e:
                feature["actions"].append({
                    "name": f"작업 생성 실패: {e}",
                    "startDate": "2025-01-01T00:00:00",
                    "endDate": "2025-01-01T23:59:59",
                    "importance": 0
                })

    return json.dumps(data, ensure_ascii=False, indent=2)

In [115]:
parse_and_generate_all(workspace_data)  # test

'{\n  "workspaceId": 9,\n  "recommendedCategories": [\n    {\n      "categoryId": 4,\n      "name": "게시판 기능",\n      "features": [\n        {\n          "featureId": 75,\n          "name": "게시글 관리",\n          "actions": [\n            {\n              "name": "게시글 신고",\n              "importance": 1,\n              "startDate": "2025-06-19T14:25:34.094132",\n              "endDate": "2025-06-21T14:25:34.094132"\n            },\n            {\n              "name": "게시글 검색",\n              "importance": 4,\n              "startDate": "2025-06-20T14:25:34.094135",\n              "endDate": "2025-07-04T14:25:34.094135"\n            },\n            {\n              "name": "작업 생성 실패: Missing some input keys: {\'agent_scratchpad\', \'tools\', \'tool_names\'}",\n              "startDate": "2025-01-01T00:00:00",\n              "endDate": "2025-01-01T23:59:59",\n              "importance": 0\n            }\n          ]\n        }\n      ]\n    }\n  ]\n}'

In [116]:
# 툴 정의
parse_tool = Tool(
    name="ParseWorkflow",
    func=parse_and_generate_all,
    description="JSON 작업 흐름에 대해 feature별로 다음 추천 작업(actions) 3개를 추가합니다."
)

final_tool = Tool(
    name="Final Answer",
    func=lambda x: x,  # 결과를 그대로 반환
    description="최종 JSON 결과를 반환합니다."
)

tools = [parse_tool, final_tool]

In [117]:
type(formatted_input)  # test
print(formatted_input)

# LangChain ReAct Agent 프롬프트 (JSON 작업 흐름 기반 액션 추천용)

당신은 JSON 작업 흐름을 분석하여, 각 기능(feature)에 대해 이후에 자연스럽게 이어질 작업(action) 3개씩을 추천하는 에이전트입니다.

다음은  기능의 기존 작업들입니다:



각 추천 작업은 반드시 다음 네 가지 필드를 포함해야 합니다:
- `name`: 작업 이름
- `startDate`: 시작 시점 (ISO 8601 형식, 예: YYYY-MM-DDTHH:MM:SS)
- `endDate`: 종료 시점 (startDate보다 늦어야 함)
- `importance`: 중요도 (1~5 사이의 정수, 값이 클수록 중요한 작업임)

결과는 반드시 **순수 JSON 형식**으로 출력해야 하며, **설명, 마크다운, 부가 텍스트 없이** 결과만 보여주어야 합니다.

응답은 아래와 같은 형식이어야 합니다:

```json
{
  "workspaceId": "",
  "recommendedCategories": [
    {
      "categoryId": "",
      "name": "",
      "startDate": "YYYY-MM-DDTHH:MM:SS",
      "endDate": "YYYY-MM-DDTHH:MM:SS",
      "importance": 5,
      "features": [
        {
          "featureId": "",
          "name": "",
          "startDate": "YYYY-MM-DDTHH:MM:SS",
          "endDate": "YYYY-MM-DDTHH:MM:SS",
          "importance": 1,
          "actions": [
            {
              "name": "",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDat

In [118]:
# 프롬프트 객체 생성
# prompt = PromptTemplate(
#     template=RCMD_PROMPT,
#     input_variables=["ws_data", "tools", "tool_names", "agent_scratched"]
# )

# 먼저 에이전트 생성하고
# create_react_agent()는 initialize_agent() 보다 세밀한 설정 가능
agent = create_react_agent(
    llm=llm,
    tools=tools,  # 단순 tools 선언, AgentExecutor에서 tools 재전달 필요
    prompt=prompt
)

# 다음에 AgentExecutor로 루프 반복 컨트롤?
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,  # tools의 기능 전달
    max_iterations=5,
    max_execution_time=30,
    early_stopping_method="generate",
    verbose=True
)

In [121]:
prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))  # 위에서 이미 함
prompt_filled

'# LangChain ReAct Agent 프롬프트 (JSON 작업 흐름 기반 액션 추천용)\n\n당신은 JSON 작업 흐름을 분석하여, 각 기능(feature)에 대해 이후에 자연스럽게 이어질 작업(action) 3개씩을 추천하는 에이전트입니다.\n\n다음은 {{featureName}} 기능의 기존 작업들입니다:\n\n{{existingActions}}\n\n각 추천 작업은 반드시 다음 네 가지 필드를 포함해야 합니다:\n- `name`: 작업 이름\n- `startDate`: 시작 시점 (ISO 8601 형식, 예: YYYY-MM-DDTHH:MM:SS)\n- `endDate`: 종료 시점 (startDate보다 늦어야 함)\n- `importance`: 중요도 (1~5 사이의 정수, 값이 클수록 중요한 작업임)\n\n결과는 반드시 **순수 JSON 형식**으로 출력해야 하며, **설명, 마크다운, 부가 텍스트 없이** 결과만 보여주어야 합니다.\n\n응답은 아래와 같은 형식이어야 합니다:\n\n```json\n{\n  "workspaceId": "{{workspace_Id}}",\n  "recommendedCategories": [\n    {\n      "categoryId": "{{category_Id}}",\n      "name": "{{category_name}}",\n      "startDate": "YYYY-MM-DDTHH:MM:SS",\n      "endDate": "YYYY-MM-DDTHH:MM:SS",\n      "importance": 5,\n      "features": [\n        {\n          "featureId": "{{feature_Id}}",\n          "name": "{{feature_name}}",\n          "startDate": "YYYY-MM-DDTHH:MM:SS",\n          "endDate": "YYYY-MM-DDTHH:MM:SS",\n          "imp

In [122]:
formatted_input

'# LangChain ReAct Agent 프롬프트 (JSON 작업 흐름 기반 액션 추천용)\n\n당신은 JSON 작업 흐름을 분석하여, 각 기능(feature)에 대해 이후에 자연스럽게 이어질 작업(action) 3개씩을 추천하는 에이전트입니다.\n\n다음은  기능의 기존 작업들입니다:\n\n\n\n각 추천 작업은 반드시 다음 네 가지 필드를 포함해야 합니다:\n- `name`: 작업 이름\n- `startDate`: 시작 시점 (ISO 8601 형식, 예: YYYY-MM-DDTHH:MM:SS)\n- `endDate`: 종료 시점 (startDate보다 늦어야 함)\n- `importance`: 중요도 (1~5 사이의 정수, 값이 클수록 중요한 작업임)\n\n결과는 반드시 **순수 JSON 형식**으로 출력해야 하며, **설명, 마크다운, 부가 텍스트 없이** 결과만 보여주어야 합니다.\n\n응답은 아래와 같은 형식이어야 합니다:\n\n```json\n{\n  "workspaceId": "",\n  "recommendedCategories": [\n    {\n      "categoryId": "",\n      "name": "",\n      "startDate": "YYYY-MM-DDTHH:MM:SS",\n      "endDate": "YYYY-MM-DDTHH:MM:SS",\n      "importance": 5,\n      "features": [\n        {\n          "featureId": "",\n          "name": "",\n          "startDate": "YYYY-MM-DDTHH:MM:SS",\n          "endDate": "YYYY-MM-DDTHH:MM:SS",\n          "importance": 1,\n          "actions": [\n            {\n              "name": "",\n              "startDate": "YYYY

In [119]:
# prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))  # 위에서 이미 함
response = agent_executor.invoke({"input": formatted_input})

# *** 답변 생성 확인 ***
# print(response)



> Entering new AgentExecutor chain...


KeyError: 'Input to PromptTemplate is missing variables {\'\\n  "workspaceId"\'}.  Expected: [\'\\n  "workspaceId"\', \'agent_scratchpad\'] Received: [\'input\', \'intermediate_steps\', \'agent_scratchpad\']\nNote: if you intended {\n  "workspaceId"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "workspaceId"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
response

{'input': 'Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:\n\n{\n  "workspaceId": "작업 중인 workspaceId",\n  "recommendedCategories": [\n    {\n      "categoryId": "작업 중인 categoryId",\n      "name": "카테고리 이름",\n      "startDate": "YYYY-MM-DDTHH:MM:SS",\n      "endDate": "YYYY-MM-DDTHH:MM:SS",\n      "importance": 정수,\n      "features": [\n        {\n          "featureId": "작업 중인 featureId",\n          "name": "기능 이름",\n          "startDate": "YYYY-MM-DDTHH:MM:SS",\n          "endDate": "YYYY-MM-DDTHH:MM:SS",\n          "importance": 정수,\n          "actions": [\n            {\n              "name": "추천 action",\n              "startDate": "YYYY-MM-DDTHH:MM:SS",\n              "endDate": "YYYY-MM-DDTHH:MM:SS",\n              "importance": 정수\n            },\n            {\n              "name": "추천 action",\n         